In [1]:
import numpy as np
import pandas as pd
import seaborn as sns 
import dill

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelBinarizer,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline, FeatureUnion

In [2]:
df = pd.read_csv('./weatherAUS.csv')

In [3]:
df.drop(['Date'], axis=1, inplace=True)
df.drop(['Location'], axis=1, inplace=True)

In [4]:
percent_zeros = (df.isnull().sum() / df.isnull().count())
df.drop(list(percent_zeros.loc[percent_zeros > 0.15].index), axis=1, inplace=True)
df.dropna(inplace=True)

In [5]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]
    
class MyLabelBinarizer(TransformerMixin):
    def __init__(self):
        self.encoder = LabelBinarizer()
        
    def fit(self, X, y=0):
        self.encoder.fit(X.astype(str))
        return self
    
    def transform(self, X, y=0):
        return self.encoder.transform(X.astype(str))

In [6]:
categorical_columns = ['WindGustDir','WindDir9am','WindDir3pm']
continuous_columns = df.columns.drop(categorical_columns).tolist()
continuous_columns.remove('RainToday')
continuous_columns.remove('RainTomorrow')

In [7]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col))
            ])
    final_transformers.append((cont_col, cont_transformer))
    

bin_transformer = Pipeline([
                 ('selector', FeatureSelector(column='RainToday')),
                 ('binarizer', MyLabelBinarizer())
             ])

final_transformers.append(('RainToday', bin_transformer))
    
feats = FeatureUnion(final_transformers)

In [8]:
pipeline = Pipeline([
    ('features', feats),
    ('classifier', RandomForestClassifier(criterion='gini',
                                          max_depth=10, 
                                          max_features='auto',
                                          n_estimators=200))
])

In [9]:
df['RainTomorrow'] = df['RainTomorrow'].astype(str)

lb = LabelBinarizer()

df['RainTomorrow'] = lb.fit_transform(df['RainTomorrow'])

In [10]:
y = df['RainTomorrow']
X = df.drop(['RainTomorrow'], axis=1)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [12]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('WindGustDir',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='WindGustDir')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='WindGustDir'))])),
                                                ('WindDir9am',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='WindDir9am')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='WindDir9am'))])),
                                                ('WindDir3pm',
                                                 Pipeline(steps=[('

In [13]:
pipeline.score(X_train, y_train)

0.8617531342112921

In [14]:
y_pred = pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

0.849666450203672

In [15]:
f1_score(y_test, y_pred)

0.5553130184231205

In [16]:
with open('random_forest_pipeline.dill', 'wb') as f:
    dill.dump(pipeline, f)